In [21]:
import sys
import os
import glob

package_path = "../../funROI"
sys.path.append(os.path.abspath(package_path))
import funROI

funROI.set_bids_data_folder("../data/bids")
funROI.set_bids_deriv_folder("../data/bids/derivatives/funROI")
funROI.set_bids_preprocessed_folder("../data/bids/derivatives/fmriprep")

# Turn off all warnings
import warnings
warnings.filterwarnings("ignore")

In [22]:
from funROI.analysis import SpatialCorrelationEstimator
from funROI.froi import FROI

In [23]:
subjects = [
    "040",
    "057",
    "059",
    "056",
    "067",
    "068",
    "019",
    "070",
    "087",
    "078",
    "089",
    "076",
]
task_contrasts=[
    ("SWNloc", ["S", "W", "N"]),
    ("EventsOrig", ["Sem_photo", "Perc_photo", "Sem_sent", "Perc_sent"]),
]

# Whole-Parcels Spatial Correlation

In [24]:
spatial_correlation_estimator = SpatialCorrelationEstimator(subjects, ['Events', 'Language'])
spcorr = spatial_correlation_estimator.compute_spatial_correlation(task_contrasts)
spcorr.to_csv("spatial_correlation_parcels.csv")

In [25]:
# Compare to spm
import pandas as pd
import os
funroi_parcels_spcorr = pd.read_csv("spatial_correlation_parcels.csv")
spm_parcels_spcorr = []
for parcels in ['Language', 'Events']:
    spm_parcels_spcorr_i = []
    for var1 in ['SWN', 'EventsOrig']:
        for var2 in ['SWN', 'EventsOrig']:
            spm_parcels_spcorr_i.append(pd.read_csv(os.path.join("../data/spm_ss_results/spcorr_parcels_{parcels}_{var1}2{var2}_2024-10-24/spm_bcc_data.EffectSize.csv".format(parcels=parcels, var1=var1, var2=var2))))
    spm_parcels_spcorr_i = pd.concat(spm_parcels_spcorr_i)
    spm_parcels_spcorr_i['parcels'] = parcels
    spm_parcels_spcorr.append(spm_parcels_spcorr_i)
spm_parcels_spcorr = pd.concat(spm_parcels_spcorr).reset_index(drop=True)
sorted_subjects = sorted(funroi_parcels_spcorr["subject"].unique())
spm_parcels_spcorr["Subject"] = spm_parcels_spcorr["Subject"].map(
    lambda x: sorted_subjects[x - 1]
)
spm_parcels_spcorr[['contrast1', 'contrast2']] = spm_parcels_spcorr['Effects'].str.split('/', expand=True)
spm_parcels_spcorr

,ROI,Subject,Effects,Fisher transformed correlation coefficients,parcels,contrast1,contrast2
0,1,19,S/S,1.095165,Language,S,S
1,1,19,W/S,0.973833,Language,W,S
2,1,19,N/S,0.817550,Language,N,S
3,1,19,S/W,0.999514,Language,S,W
4,1,19,W/W,0.960471,Language,W,W
...,...,...,...,...,...,...,...
9991,11,89,Perc_sent/Sem_sent,1.065478,Events,Perc_sent,Sem_sent
9992,11,89,Sem_photo/Perc_sent,0.435998,Events,Sem_photo,Perc_sent
9993,11,89,Perc_photo/Perc_sent,0.473723,Events,Perc_photo,Perc_sent
9994,11,89,Sem_sent/Perc_sent,1.065478,Events,Sem_sent,Perc_sent


In [26]:
merged = pd.merge(
    funroi_parcels_spcorr,
    spm_parcels_spcorr,
    left_on=["subject", "effect1", "effect2", 'localizer_name', "froi"],
    right_on=["Subject", "contrast1", "contrast2", 'parcels', "ROI"],
)
merged = merged[
    [
        "subject",
        "effect1",
        "effect2",
        "parcels",
        "froi",
        "fisher_z",
        "Fisher transformed correlation coefficients"
    ]
].rename(
    columns={
        "fisher_z": "funROI_fisher_z",
        "Fisher transformed correlation coefficients": "spm_ss_fisher_z",
    }
)
effect_order = ['Sem_photo', 'Perc_photo', 'Sem_sent', 'Perc_sent', 'S', 'W', 'N']
merged['effect1'] = pd.Categorical(merged['effect1'], categories=effect_order, ordered=True)
merged['effect2'] = pd.Categorical(merged['effect2'], categories=effect_order, ordered=True)

In [27]:
import numpy as np
np.max(merged['funROI_fisher_z'] - merged['spm_ss_fisher_z']), np.min(merged['funROI_fisher_z'] - merged['spm_ss_fisher_z'])

(np.float64(4.999220070134314e-07), np.float64(-4.99981518187198e-07))

# fROI Spatial Correlation

In [28]:
frois = [
    (
        "SWNloc",
        [
            FROI(
                localizer="S-N",
                thresholdType="percent",
                thresholdValue=0.1,
                parcels="Language",
            ),
        ],
    ),
    (
        "EventsOrig",
        [
            FROI(
                localizer="Sem_photominusPerc_photo",
                thresholdType="percent",
                thresholdValue=0.1,
                parcels="Events",
            ),
            FROI(
                localizer="Sem_sentminusPerc_sent",
                thresholdType="percent",
                thresholdValue=0.1,
                parcels="Events",
            ),
            FROI(
                localizer="SemMinusPercBothMax",
                contrasts=[
                    "Sem_photominusPerc_photo",
                    "Sem_sentminusPerc_sent",
                ],
                conjunctionType="max",
                thresholdType="percent",
                thresholdValue=0.1,
                parcels="Events",
            ),
        ],
    ),
]

In [29]:
spatial_correlation_estimator = SpatialCorrelationEstimator(subjects, frois)
spcorr = spatial_correlation_estimator.compute_spatial_correlation(task_contrasts)
spcorr.to_csv("spatial_correlation_frois.csv")

In [30]:
# Compare to spm
import pandas as pd
import os
funroi_frois_spcorr = pd.read_csv("spatial_correlation_frois.csv")
spm_frois_spcorr = []
for frois in ['Sem-Perc_Both', 'Sem-Perc_Sent', 'Sem-Perc_Photo', 'S-N']:
    spm_frois_spcorr_i = []
    for var1 in ['SWN', 'EventsOrig']:
        for var2 in ['SWN', 'EventsOrig']:
            spm_frois_spcorr_i.append(pd.read_csv(os.path.join("../data/spm_ss_results/spcorr_frois_{frois}_{var1}2{var2}_2024-10-24/spm_bcc_data.EffectSize.csv".format(frois=frois, var1=var1, var2=var2))))
    spm_frois_spcorr_i = pd.concat(spm_frois_spcorr_i)
    spm_frois_spcorr_i['localizer'] = frois
    spm_frois_spcorr.append(spm_frois_spcorr_i)
spm_frois_spcorr = pd.concat(spm_frois_spcorr).reset_index(drop=True)
sorted_subjects = sorted(funroi_frois_spcorr["subject"].unique())
spm_frois_spcorr["Subject"] = spm_frois_spcorr["Subject"].map(
    lambda x: sorted_subjects[x - 1]
)
spm_frois_spcorr[['contrast1', 'contrast2']] = spm_frois_spcorr['Effects'].str.split('/', expand=True)
spm_frois_spcorr['localizer'] = spm_frois_spcorr['localizer'].map({
    'Sem-Perc_Both': 'SemMinusPercBothMax',
    'Sem-Perc_Sent': 'Sem_sentminusPerc_sent',
    'Sem-Perc_Photo': 'Sem_photominusPerc_photo',
    'S-N': 'S-N'
})
spm_frois_spcorr

,ROI,Subject,Effects,Fisher transformed correlation coefficients,localizer,contrast1,contrast2
0,1,19,S/S,0.355252,SemMinusPercBothMax,S,S
1,1,19,W/S,0.392188,SemMinusPercBothMax,W,S
2,1,19,N/S,0.210841,SemMinusPercBothMax,N,S
3,1,19,S/W,0.376171,SemMinusPercBothMax,S,W
4,1,19,W/W,0.541497,SemMinusPercBothMax,W,W
...,...,...,...,...,...,...,...
22927,6,89,Perc_sent/Sem_sent,1.470504,S-N,Perc_sent,Sem_sent
22928,6,89,Sem_photo/Perc_sent,1.392418,S-N,Sem_photo,Perc_sent
22929,6,89,Perc_photo/Perc_sent,1.472175,S-N,Perc_photo,Perc_sent
22930,6,89,Sem_sent/Perc_sent,1.470504,S-N,Sem_sent,Perc_sent


In [31]:
merged = pd.merge(
    funroi_frois_spcorr,
    spm_frois_spcorr,
    left_on=["subject", "effect1", "effect2", 'localizer_name', "froi"],
    right_on=["Subject", "contrast1", "contrast2", 'localizer', "ROI"],
)
merged = merged[
    [
        "subject",
        "effect1",
        "effect2",
        "localizer",
        "froi",
        "fisher_z",
        "Fisher transformed correlation coefficients"
    ]
].rename(
    columns={
        "fisher_z": "funROI_fisher_z",
        "Fisher transformed correlation coefficients": "spm_ss_fisher_z",
    }
)
effect_order = ['Sem_photo', 'Perc_photo', 'Sem_sent', 'Perc_sent', 'S', 'W', 'N']
merged['effect1'] = pd.Categorical(merged['effect1'], categories=effect_order, ordered=True)
merged['effect2'] = pd.Categorical(merged['effect2'], categories=effect_order, ordered=True)

In [32]:
import numpy as np
np.max(merged['funROI_fisher_z'] - merged['spm_ss_fisher_z']), np.min(merged['funROI_fisher_z'] - merged['spm_ss_fisher_z'])

(np.float64(0.0066763136347609), np.float64(-0.02289919371971727))

In [33]:
# top absolute differences
merged['abs_diff'] = np.abs(merged['funROI_fisher_z'] - merged['spm_ss_fisher_z'])
merged.loc[merged['abs_diff']>1e-6].sort_values("abs_diff", ascending=False)

,subject,effect1,effect2,localizer,froi,funROI_fisher_z,spm_ss_fisher_z,abs_diff
14381,70,Sem_photo,Sem_photo,Sem_sentminusPerc_sent,4,0.408178,0.431077,0.022899
14389,70,Perc_photo,Perc_photo,Sem_sentminusPerc_sent,4,0.069734,0.081677,0.011943
14370,70,W,Perc_sent,Sem_sentminusPerc_sent,4,-0.037607,-0.025862,0.011745
14400,70,Perc_sent,W,Sem_sentminusPerc_sent,4,-0.037607,-0.025862,0.011745
14401,70,Perc_sent,N,Sem_sentminusPerc_sent,4,0.042589,0.054202,0.011613
14377,70,N,Perc_sent,Sem_sentminusPerc_sent,4,0.042589,0.054202,0.011613
14397,70,Sem_sent,Sem_sent,Sem_sentminusPerc_sent,4,1.109874,1.119983,0.010109
14399,70,Perc_sent,S,Sem_sentminusPerc_sent,4,-0.042692,-0.033660,0.009032
14363,70,S,Perc_sent,Sem_sentminusPerc_sent,4,-0.042692,-0.033660,0.009032
14383,70,Sem_photo,Sem_sent,Sem_sentminusPerc_sent,4,0.232244,0.240059,0.007815
